Link to GitHub: https://github.com/Stampe04/CSS_Assignment1.git

All group members contributed equally to this assignment

### Part 1: Ready Made vs Custom Made Data ###

***What are pros and cons of the custom-made data used in Centola's experiment (the first study presented in the lecture) and the ready-made data used in Nicolaides's study (the second study presented in the lecture)? You can support your arguments based on the content of the lecture and the information you read in Chapter 2.3 of the book (answer in max 150 words).***

...paste our answer from the drive

***How do you think these differences can influence the interpretation of the results in each study? (answer in max 150 words)***

...paste our answer from the drive

### Part 2: Find Researchers using the OpenAlex API ###

In [2]:
#TODO: Insert code for D1 here

***Which challenges did you encounter? How did you address them?***

In the beggining, we had some issues with the API itself, since we found the API documentation very confusing to navigate. However, this slowly became less of an issue as we went along and got more familiar with the documentation and the code. Furthermore, We had some initial issues loading the csv file properly, but this was solved by doing some simple research on handling csv files in Python. In addition, we ran into trouble with our API keys multiple times, as we kept running into issues with having enough tokens per day. We solved this by testing our code on a limited amount of data to use our keys sparingly, and then do the full run once we knew the code worked. 

***Choose one problem you faced while collecting the data and describe your solution. Why did you choose this approach, and what impact might it have on your data?***

Insert answer here...

### Part 3: Collect Research Articles ###

In [3]:
#TODO: Insert code for D2 and D3 here

***How many works are listed in your Dataset D2 (IC2S2 papers) dataframe? How many unique researchers have co-authored these works?***

This is also answerd in the cell above. There are 1367 works in D2 and the number of unique authors that have co-authored are 2284

***Describe the strategies you implemented to make your code more efficient. How did your approach affect your code's execution time?***

We used a limit of "max_authors" to make sure we had enough API requests, as we have had to deal with that problem a lot. The filter for citations makes sure that OpenAlex only takes "more relevant" articles, which speeds up the process significantly. The "author_count" filter makes sure we only focus on "smaller collaborations" and don't focus on very large ones. The filter regarding "eligible authors" also filters out a lot, as it focuses on authors that have contributed a significant amount, which helps to signify their significance.

***Reflect on the rationale behind setting specific thresholds for the total number of works by an author, the citation count, the number of authors per work, and the relevance of works to specific fields. How do these filtering criteria contribute to the relevance of the dataset you compiled? Do you believe any aspects of Computational Social Science research might be underrepresented or overrepresented as a result of these choices?***

Before applying any of the filters or specific settings, we ran a testing round on just one person, which alone gave us an enormous output, and it was very difficult to find specific information in it. This alone made us realize how necessary it would be to apply filters when searching for multiple people, but in practice, we also realised just how powerful of a tool it can be, especially when there is very specific information you are interested in or a specific criteria or threshold you want to limit yourself to. Overall, we think they are an excellent tool.  

However, we also see some potential problem when applying these filters. As an example, if you choose to filter based on a work or an author that have been cited at least x amount of times, you may underrepresent authors or works that are legitimately good but have just not been used much or discovered yet. Moreover, this can overrepresent works or authors that may not be good anymore or have become outdated, just because they have been cited a lot. 